In [5]:
import pandas as pd
import geopandas as gpd

In [6]:
gdf = gpd.read_file('datos/municipios_2020_INEGI.geojson')
gdf.head(1)

,CVEGEO,NOM_ENT,NOMGEO,pobTotal2020,cve_cont,Estado,Municipio,Localidades afectadas,Localidades sin acceso,geometry
0,13068,Hidalgo,Tianguistengo,14340.0,Hidalgo Tianguistengo,Hidalgo,Tianguistengo,30,14,"MULTIPOLYGON (((-98.50451 20.88541, -98.50409 ..."


In [10]:
gdf['cve_cont'] = gdf['NOM_ENT'] + ' ' + gdf['NOMGEO']
gdf.head(1)

,CVEGEO,NOM_ENT,NOMGEO,pobTotal2020,geometry,cve_cont
0,13001,Hidalgo,Acatlán,22268.0,"MULTIPOLYGON (((-98.46896 20.33477, -98.46877 ...",Hidalgo Acatlán


In [30]:
df = pd.read_excel('datos/captura_información_afectaciones_lluvia.xlsx', sheet_name='trabajo')
print(df.shape)
df.head(1)

(109, 4)


,Estado,Municipio,Localidades afectadas,Localidades sin acceso
0,Hidalgo,Tianguistengo,30.0,14.0


In [32]:
# Corregimos municipios
df['Municipio'] = df['Municipio'].replace({
    'Xicotepec de Juárez': 'Xicotepec',
    'Cadereyta De Montes': 'Cadereyta de Montes',
    'Tampacan':'Tampacán',
    'Tanquian de Escobedo':'Tanquián de Escobedo',
    'Poza Rica':'Poza Rica de Hidalgo',
})

In [33]:
df['cve_cont'] = df['Estado'] + ' ' + df['Municipio']
df.head(1)

,Estado,Municipio,Localidades afectadas,Localidades sin acceso,cve_cont
0,Hidalgo,Tianguistengo,30.0,14.0,Hidalgo Tianguistengo


In [44]:
gdf_u = pd.merge(gdf,df, on='cve_cont', how='right', indicator=True)
print(gdf_u.shape)
gdf_u.head(1)

(109, 11)


,CVEGEO,NOM_ENT,NOMGEO,pobTotal2020,geometry,cve_cont,Estado,Municipio,Localidades afectadas,Localidades sin acceso,_merge
0,13068,Hidalgo,Tianguistengo,14340.0,"MULTIPOLYGON (((-98.50451 20.88541, -98.50409 ...",Hidalgo Tianguistengo,Hidalgo,Tianguistengo,30.0,14.0,both


In [45]:
# Filtrar los registros que no se unieron (es decir, que solo están en df)
faltantes = gdf_u[gdf_u['_merge'] == 'right_only']
print(faltantes.shape)
faltantes

(0, 11)


,CVEGEO,NOM_ENT,NOMGEO,pobTotal2020,geometry,cve_cont,Estado,Municipio,Localidades afectadas,Localidades sin acceso,_merge


In [40]:
gdf_u.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 109 entries, 0 to 108
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   CVEGEO                  109 non-null    object  
 1   NOM_ENT                 109 non-null    object  
 2   NOMGEO                  109 non-null    object  
 3   pobTotal2020            109 non-null    float64 
 4   geometry                109 non-null    geometry
 5   cve_cont                109 non-null    object  
 6   Estado                  109 non-null    object  
 7   Municipio               109 non-null    object  
 8   Localidades afectadas   109 non-null    object  
 9   Localidades sin acceso  109 non-null    object  
 10  _merge                  109 non-null    category
dtypes: category(1), float64(1), geometry(1), object(8)
memory usage: 9.6+ KB


# simplificar las geometrias

In [8]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [9]:
gdf.to_crs(epsg=6372, inplace=True)
gdf_simplificada = gdf.copy()
gdf_simplificada['geometry'] = gdf_simplificada['geometry'].simplify(200, preserve_topology=True)
gdf_simplificada.to_crs(epsg=4326, inplace=True)

In [12]:
gdf_simplificada.head()

,CVEGEO,NOM_ENT,NOMGEO,pobTotal2020,cve_cont,Estado,Municipio,Localidades afectadas,Localidades sin acceso,geometry
0,13068,Hidalgo,Tianguistengo,14340.0,Hidalgo Tianguistengo,Hidalgo,Tianguistengo,30,14,"POLYGON ((-98.50451 20.88541, -98.49998 20.883..."
1,13053,Hidalgo,San Bartolo Tutotepec,17699.0,Hidalgo San Bartolo Tutotepec,Hidalgo,San Bartolo Tutotepec,12,10,"POLYGON ((-98.10904 20.61589, -98.10052 20.614..."
2,13027,Hidalgo,Huehuetla,22846.0,Hidalgo Huehuetla,Hidalgo,Huehuetla,67,6,"POLYGON ((-98.06778 20.67381, -98.03998 20.631..."
3,13073,Hidalgo,Tlanchinol,37722.0,Hidalgo Tlanchinol,Hidalgo,Tlanchinol,16,6,"POLYGON ((-98.70617 21.18701, -98.70212 21.185..."
4,13025,Hidalgo,Huautla,20673.0,Hidalgo Huautla,Hidalgo,Huautla,15,6,"POLYGON ((-98.21558 21.15008, -98.21683 21.144..."


In [13]:
gdf_simplificada = gdf_simplificada.rename(columns={
    'Localidades afectadas': 'locAfectada',
    'Localidades sin acceso': 'locSinAcceso'
})

In [14]:
gdf_simplificada.head(1)

,CVEGEO,NOM_ENT,NOMGEO,pobTotal2020,cve_cont,Estado,Municipio,locAfectada,locSinAcceso,geometry
0,13068,Hidalgo,Tianguistengo,14340.0,Hidalgo Tianguistengo,Hidalgo,Tianguistengo,30,14,"POLYGON ((-98.50451 20.88541, -98.49998 20.883..."


In [15]:
#gdf_simplificada.drop(columns=['_merge'], inplace=True)

gdf_simplificada.to_file('datos/municipios_2020_INEGI_v2.geojson', driver='GeoJSON')

c:\Users\COMIMSA\miniconda3\envs\gds\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
